In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql.window import Window as W

spark = SparkSession.builder.appName('Puzzle').getOrCreate()

In [2]:
df = spark.read.csv('data/puzzle_041.csv', header=True, inferSchema=True)
df.show()

+-------+-----+--------+--------------+-------+
|deal_id|price|quantity|total_quantity| months|
+-------+-----+--------+--------------+-------+
|      2|  4.0|   10000|        310000|00:00.0|
|      2|  4.0|   10000|        280000|00:00.0|
|      2|  4.5|   10000|        310000|00:00.0|
|      2|  4.0|   10000|        310000|00:00.0|
|      2|  4.0|   15000|        280000|00:00.0|
|      2|  4.0|   15000|        310000|00:00.0|
+-------+-----+--------+--------------+-------+



In [24]:
df.select(
    '*',
    F.when(F.lag('price',1,0).over(W.orderBy(F.lit(1))) !=F.col('price'),1).otherwise(0).alias('ss'),
    ).select('*',
        F.sum(F.col('ss')).over(W.orderBy(F.col('deal_id'))).alias('dd')).show()

+-------+-----+--------+--------------+-------+---+---+
|deal_id|price|quantity|total_quantity| months| ss| dd|
+-------+-----+--------+--------------+-------+---+---+
|      2|  4.0|   10000|        310000|00:00.0|  1|  3|
|      2|  4.0|   10000|        280000|00:00.0|  0|  3|
|      2|  4.5|   10000|        310000|00:00.0|  1|  3|
|      2|  4.0|   10000|        310000|00:00.0|  1|  3|
|      2|  4.0|   15000|        280000|00:00.0|  0|  3|
|      2|  4.0|   15000|        310000|00:00.0|  0|  3|
+-------+-----+--------+--------------+-------+---+---+

